## Precarga de librerias y funciones

In [76]:
%%capture
%run "../../recurrentes.ipynb"

In [77]:
%%capture
%run "../../funciones.ipynb"

#### Excel

In [78]:
# Activar Excel de seguimiento de medidas? 

excel = True
if excel == True:
    # Ingresar path excel para anotar resultados
    path_excel = base_path_l + r'\resultados_backtesting.xlsx'
    date = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
    

In [79]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) competencia_03_ct_sin_psi.parquet

dataset = dataset_clase_ternaria_l
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


## Formateo pre modelo

In [80]:

# [202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106] modelo 10 meses

ganancia_acierto = 273000
costo_estimulo = 7000

#Defenir mes de train y test

mes_train = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
        201909, 201910, 201911, 201912, 202001, 202002,202101, 202102, 202103, 202104, 202105]
mes_test = 202107


# Defenir mejores parámetros para el modelo

best_params = {'num_leaves': 7403,  'learning_rate': 0.04206182379229533,  'min_data_in_leaf': 191, 'feature_fraction': 0.4078072640925111,  'bagging_fraction': 0.300397113625344}
best_iter = 104

In [81]:
# predecir_punto_de_corte(  
#     dataset,  
#     mes_de_entrenamiento,  
#     mes_de_prueba,  
#     best_params,  
#     best_iteration,  
#     rango_cortes,  
#     semillas,  


In [82]:

data = df_train
data['tmobile_app'] = pd.to_numeric(data['tmobile_app'], errors='coerce')  # Convierte no numéricos a NaN
data['tmobile_app'] = data['tmobile_app'].fillna(0).astype('bool')  # Llena los NaN y convierte a booleano


data['cmobile_app_trx'] = pd.to_numeric(data['cmobile_app_trx'], errors='coerce')  # Convierte no numéricos a NaN
data['cmobile_app_trx'] = data['cmobile_app_trx'].fillna(0).astype('bool')  # Llena los NaN y convierte a booleano

In [83]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)
    
# Binarización de la variable target

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='BAJA+2', 1, 0)

# Train-Test Split

if isinstance(mes_train, list):
    df_train = data[data['foto_mes'].isin(mes_train)]
    
else: df_train = data[data['foto_mes']<=mes_train]

df_test = data[data['foto_mes']==mes_test]

# definiendo X e Y

clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

In [84]:
df_train['foto_mes'].unique(), df_test['foto_mes'].unique()

(array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
        201909, 201910, 201911, 201912, 202001, 202002, 202101, 202102,
        202103, 202104, 202105], dtype=int64),
 array([202107], dtype=int64))

## Back-testing

In [85]:
respuesta = 'personalizado'
if respuesta == '':
   input (" 'personalizado' o 'default': ")
   if respuesta == 'personalizado':
      
      best_params = {'num_leaves': '',
      'learning_rate': '',
      'min_data_in_leaf': '',
      'feature_fraction': '',
      'bagging_fraction': ''}
   elif respuesta == 'default':
      params = {}


In [86]:
best_params = {'num_leaves': 2359,  'learning_rate': 0.005451837118174039,  'min_data_in_leaf': 6125, 'feature_fraction': 0.31138983413387283, 'bagging_fraction': 0.5231468637859576}
best_iter = 1619

In [87]:
X_train['tmobile_app']

0          False
1          False
2          False
3          False
4          False
           ...  
4901223    False
4901224    False
4901225    False
4901226    False
4901232    False
Name: tmobile_app, Length: 2682685, dtype: bool

In [88]:
df_voting = pd.DataFrame()


train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)

for x in semillas:

    if respuesta == 'personalizado':
        params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'first_metric_only': True,
            'boost_from_average': True,
            'feature_pre_filter': False,
            'max_bin': 31,
            'num_leaves': best_params['num_leaves'],
            'learning_rate': best_params['learning_rate'],
            'min_data_in_leaf': best_params['min_data_in_leaf'],
            'feature_fraction': best_params['feature_fraction'],
            'bagging_fraction': best_params['bagging_fraction'],
            'seed': x,  # Usamos el x actual del bucle
            'verbose': 0
        }

    elif respuesta == 'default':
        params = {
            'seed': x  # Si deseas usar x también en el caso 'default'
        }

    # Entrenar el modelo con los parámetros actualizados
    model = lgb.train(params,
                      train_data,
                      num_boost_round=best_iter)

    y_pred = model.predict(X_test)
    df_voting[f'prediccion_seed_{x}'] = y_pred

df_voting['prediccion'] = df_voting.mean(axis=1)
df_voting.index = X_test.index


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [89]:
prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
prediccion.shape 

(165152, 2)

In [90]:
prediccion.isna().sum()

numero_de_cliente    0
probabilidad         0
dtype: int64

In [91]:
df_train['foto_mes'].unique()

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202101, 202102,
       202103, 202104, 202105], dtype=int64)

In [92]:
df_test = df_test.reindex(X_test.index)


In [93]:
bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
bajas.shape

(165152, 2)

In [94]:
X_test.shape

(165152, 154)

In [95]:
print(df_test['clase_ternaria'].isna().sum())


0


In [96]:
df_test.shape

(165152, 157)

In [97]:
bajas.isna().sum()

numero_de_cliente    0
clase_ternaria       0
dtype: int64

In [98]:
prediccion.shape
merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')

merged_data.columns
merged_data_sorted = merged_data.sort_values('probabilidad', ascending=False)


# Ordenar por probabilidad de mayor a menor

# Seleccionar top clientes
top_clients = merged_data_sorted.iloc[:12000].copy()  # Trabajar con una copia

# Convertir 'clase_ternaria' a variable binaria para los top clientes
top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

top_clients.value_counts('clase_ternaria')


clase_ternaria
CONTINUA    10330
BAJA+2        880
BAJA+1        790
Name: count, dtype: int64

In [99]:
top_clients.value_counts('bajas_reales')

bajas_reales
0    11120
1      880
Name: count, dtype: int64